In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import TensorBoard

In [2]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(device=gpu, enable=True)

In [3]:
#下载mnist 数据集
mnist = keras.datasets.mnist
train_data, test_data = mnist.load_data(path='mnist.npz')
x_train, y_train = train_data
x_test, y_test = test_data
#x_train = x_train/255.
#x_test = x_test/255.
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

#改变numpy维度，加入通道维
x_train = x_train[:, :, :, np.newaxis]
x_test = x_test[:, :, :, np.newaxis]
print(x_train.shape)
print(x_test.shape)
#先对图片大小进行插值，要不太小了
#plt.imshow(x_train[0])

x_train = tf.image.resize(x_train, [200, 200])
x_test = tf.image.resize(x_test, [200, 200])
print(x_train.shape)
print(x_test.shape)

img_height, img_width = x_train.shape[1:3]
print(img_height)
print(img_width)
#利用上面的插值，都变为200*200，然后利用一个1*1*3的conv变成三通道，然后通到resnet

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000, 200, 200, 1)
(10000, 200, 200, 1)
200
200


In [4]:
#定义模型超参数
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories='auto')
y_train = np.array([y_train]).T
y_test = np.array([y_test]).T
print(y_train.shape)
print(y_test.shape)

enc.fit(y_train)
y_train = enc.transform(y_train).toarray()
y_test = enc.transform(y_test).toarray()
num_classes = enc.categories_[0].shape[0]
print(num_classes)

(60000, 1)
(10000, 1)
10


In [5]:
tbCallBack = TensorBoard(log_dir='./resnet_v1_logs',  # log 目录
                 histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算
#                  batch_size=32,     # 用多大量的数据计算直方图
                 write_graph=True,  # 是否存储网络结构图
                 write_images=True,# 是否可视化参数
                 embeddings_freq=1, 
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)

In [6]:
#在这里建立模型

inputs = Input(shape=x_train.shape[1:4])
resnet_input = Conv2D(filters=3, kernel_size=(1*1), strides=(1*1), padding='same')(inputs)
resnet_model = ResNet50(input_tensor=resnet_input, weights=None, include_top=False, input_shape=(img_height, img_width, 3))
x = resnet_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)
adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, batch_size=32, callbacks=[tbCallBack],
         validation_data=(x_test, y_test))

#resnet用法参考：https://github.com/priya-dwivedi/Deep-Learning/blob/master/resnet_keras/Residual_Network_Keras.ipynb
#resnet paper : http://openaccess.thecvf.com/content_cvpr_2016/papers/He_Deep_Residual_Learning_CVPR_2016_paper.pdf

model.save('./resnet_v1_mnist_model.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 335s 6ms/sample - loss: 0.3604 - accuracy: 0.9056 - val_loss: 0.1273 - val_accuracy: 0.9644
Epoch 2/20
60000/60000 [==============================] - 320s 5ms/sample - loss: 0.0851 - accuracy: 0.9769 - val_loss: 0.1143 - val_accuracy: 0.9698
Epoch 3/20
60000/60000 [==============================] - 319s 5ms/sample - loss: 0.0648 - accuracy: 0.9820 - val_loss: 0.1147 - val_accuracy: 0.9728
Epoch 4/20
60000/60000 [==============================] - 321s 5ms/sample - loss: 0.0532 - accuracy: 0.9845 - val_loss: 0.0550 - val_accuracy: 0.9842
Epoch 5/20
60000/60000 [==============================] - 324s 5ms/sample - loss: 0.0469 - accuracy: 0.9864 - val_loss: 0.4214 - val_accuracy: 0.8975
Epoch 6/20
60000/60000 [==============================] - 323s 5ms/sample - loss: 0.0377 - accuracy: 0.9891 - val_loss: 0.0494 - val_accuracy: 0.9870
Epoch 7/20
60000/60000 [==========================

In [7]:
preds = model.evaluate(x_test, y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Loss = 0.017265063959803772
Test Accuracy = 0.9947


In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 200, 200, 3)  6           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
______________________________________________________________________________________________

In [9]:
del model

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('resnet_v1_mnist_model.h5')

In [ ]:
preds = model.evaluate(x_test, y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))